In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
df = pd.read_csv(r"C:\Users\HP\Downloads\Medicinal Plant identification\Medicinal Plant identification\medicinal_plants_single_disease.csv")


In [4]:
df['Content'] = df['Disease Cured'] + " " + df['Medicinal Benefits']


In [5]:
df = df.dropna(subset=['Content'])


In [6]:
df['Content'] = df['Content'].fillna('')


In [7]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['Content'])

In [8]:

# Initialize the TfidfVectorizer and fit-transform the 'Disease Cured' column
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df['Disease Cured'])

def recommend_plant(disease_query):
    # Convert the disease query into a TF-IDF vector
    query_tfidf = tfidf.transform([disease_query])
    
    # Compute the cosine similarities between the query and all plants
    cosine_similarities = cosine_similarity(query_tfidf, tfidf_matrix).flatten()
    
    # Get the index of the plant with the highest cosine similarity score
    best_match_idx = cosine_similarities.argmax()
    
    # Display the best matching plant's information
    plant_name = df.iloc[best_match_idx]['Plant Name']
    disease_cured = df.iloc[best_match_idx]['Disease Cured']
    benefits = df.iloc[best_match_idx]['Medicinal Benefits']
    similarity_score = cosine_similarities[best_match_idx]
    
    print(f"Best recommendation for '{disease_query}':\n")
    print(f"Plant Name: {plant_name}")
    print(f"Disease Cured: {disease_cured}")
    print(f"Benefits: {benefits}")
    print(f"Similarity Score: {similarity_score:.2f}")
    print("-" * 40)

# Take user input for the disease or symptom
disease_query = input("Enter the disease or symptom for recommendation: ")
recommend_plant(disease_query)


Enter the disease or symptom for recommendation: cough
Best recommendation for 'cough':

Plant Name: Tulsi
Disease Cured: Cough,cold
Benefits: Tulsi boosts immunity,reduces stress improves respiratory health.
Similarity Score: 0.71
----------------------------------------


In [9]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Prepare dataset for supervised learning, creating labels based on plants
df['Label'] = df['Plant Name'].astype('category').cat.codes
X_train, X_test, y_train, y_test = train_test_split(df['Disease Cured'], df['Label'], test_size=0.2, random_state=42)

In [10]:
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train).toarray()
X_test_tfidf = vectorizer.transform(X_test).toarray()

In [11]:
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(X_train_tfidf.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(df['Label'].unique()), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_tfidf, y_train, epochs=10, validation_data=(X_test_tfidf, y_test))

# Save the trained model to .h5 format
model.save("plant_recommendation_model.h5")



Epoch 1/10


1/1 [==============================] - 3s 3s/step - loss: 3.4961 - accuracy: 0.0000e+00 - val_loss: 3.5059 - val_accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 0s 117ms/step - loss: 3.4807 - accuracy: 0.0000e+00 - val_loss: 3.5107 - val_accuracy: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 0s 111ms/step - loss: 3.4653 - accuracy: 0.0385 - val_loss: 3.5157 - val_accuracy: 0.0000e+00
Epoch 4/10
1/1 [==============================] - 0s 115ms/step - loss: 3.4499 - accuracy: 0.0385 - val_loss: 3.5207 - val_accuracy: 0.0000e+00
Epoch 5/10
1/1 [==============================] - 0s 98ms/step - loss: 3.4346 - accuracy: 0.0769 - val_loss: 3.5258 - val_accuracy: 0.0000e+00
Epoch 6/10
1/1 [==============================] - 0s 112ms/step - loss: 3.4193 - accuracy: 0.0769 - val_loss: 3.5310 - val_accuracy: 0.0000e+00
Epoch 7/10
1/1 [==============================] - 0s 111ms/step - loss: 3.4041 - accuracy: 0.1154 - val_loss: 3.5363 - val_accur

C:\Users\HP\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [12]:
# Load the model
model = tf.keras.models.load_model("plant_recommendation_model.h5")

# Convert the model to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model
with open("plant_recommendation_model.tflite", "wb") as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: C:\Users\HP\AppData\Local\Temp\tmpzm_afe79\assets


INFO:tensorflow:Assets written to: C:\Users\HP\AppData\Local\Temp\tmpzm_afe79\assets
